# Initial Setup

In [2]:
# This is needed only for the purpose of the notebook
!pip install ipytest


[notice] A new release of pip is available: 24.2 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Importing required libraries
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets

import pytest
import ipytest
ipytest.autoconfig()

In [4]:
# Getting the data
iris = datasets.load_iris()

In [5]:
# Simple setup in the data
iris_df = pd.DataFrame(iris.data, columns = iris.feature_names)
iris_df['target'] = iris.target

### Quick check in the data

In [6]:
# Quick overview of the first 5 rows in the data
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
# Simple counts in our target variable
iris_df['target'].value_counts()

target
0    50
1    50
2    50
Name: count, dtype: int64

In [8]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


### Setting up the classes to build a simple model

In [9]:
class SimplePipeline:
    def __init__(self):
        self.frame = None
        # Each value is None when we instantiate the class
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Loading the dataset, and make the train, test, split."""
        dataset = datasets.load_iris()
        
        # Removing the units (cm) from the headers
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        self.frame['target'] = dataset.target
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Execution method for running the pipeline several times."""
        self.load_dataset()
        self.train()

### Running the pipeline

In [16]:
pipeline = SimplePipeline()
pipeline.run_pipeline()
accuracy_score = pipeline.get_accuracy()
print(f'The Accuracy of the model is: {accuracy_score}')

The Accuracy of the model is: 0.9693877551020408


d:\Training\ITESM\MLOps_ITESM\practice3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


# Testing

In [15]:
# Defining the schema
iris_schema = {
    'sepal length': {
        'range': {
            'min': 4.0,
            'max': 8.0
        },
        'dtype': float,
    },
    'sepal width': {
        'range': {
            'min': 1.0,
            'max': 5.0
        },
        'dtype': float,
    },
    'petal length': {
        'range': {
            'min': 1.0,
            'max': 7.0
        },
        'dtype': float,
    },
    'petal width': {
        'range': {
            'min': 0.1,
            'max': 3.0
        },
        'dtype': float,
    }
}

In [12]:
@pytest.fixture
def pipeline():
    pl = SimplePipeline()
    pl.run_pipeline()
    return pl

### Creating the tests

In [13]:
%%ipytest

def test_input_data_ranges(pipeline):
    # Getting the maximum and minimum values for each column
    max_values = pipeline.frame.max()
    min_values = pipeline.frame.min()
    
    # Ensuring that the maximum and minimum values fall into the expected range
    for feature in pipeline.feature_names:
        assert max_values[feature] <= iris_schema[feature]['range']['max']
        assert min_values[feature] >= iris_schema[feature]['range']['min']

def test_input_data_types(pipeline):
    # Getting the data types from each column
    data_types = pipeline.frame.dtypes
    
    # Testing compatibility between data types
    for feature in pipeline.feature_names:
        assert data_types[feature] == iris_schema[feature]['dtype']

..                                                                                           [100%]
======================================== warnings summary =========================================
notebooks/Testing/t_1e69149d3fd1463583e06a69e5fffdf3.py::test_input_data_ranges
notebooks/Testing/t_1e69149d3fd1463583e06a69e5fffdf3.py::test_input_data_types
  d:\Training\ITESM\MLOps_ITESM\practice3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
2 passed, 2 warnings in 0.11s


# Let´s make the test fail

In [14]:
%%ipytest


def test_input_data_ranges(pipeline):
    max_values = pipeline.frame.max()
    min_values = pipeline.frame.min()
    
    for feature in pipeline.feature_names:
        # We change the values so the new ones are not the same as those we have already in the schema
        assert max_values[feature] < 0  # This will make the test fail
        assert min_values[feature] > 1000  # This one as well

def test_input_data_types(pipeline):
    # Getting the data types from each column
    data_types = pipeline.frame.dtypes
    
    # Making comparissons between data types
    for feature in pipeline.feature_names:
        assert data_types[feature] != iris_schema[feature]['dtype']

FF                                                                                           [100%]
============================================ FAILURES =============================================
_____________________________________ test_input_data_ranges ______________________________________

pipeline = <__main__.SimplePipeline object at 0x000001A7124C0550>

    def test_input_data_ranges(pipeline):
        max_values = pipeline.frame.max()
        min_values = pipeline.frame.min()
    
        for feature in pipeline.feature_names:
            # Cambiar los valores esperados para que no coincidan con los reales
>           assert max_values[feature] < 0  # Esto probablemente hará fallar la prueba
E           assert np.float64(7.9) < 0

C:\Users\richv\AppData\Local\Temp\ipykernel_2368\2393645784.py:7: AssertionError
______________________________________ test_input_data_types ______________________________________

pipeline = <__main__.SimplePipeline object at 0x000001A712682C90